In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

In [2]:
kerasModels = {
    'mlp': 'serialized/keras/mlpModel.keras',
    'lstm': 'serialized/keras/lstmModel.keras',
    'dnn': 'serialized/keras/dnnModel.keras'
}

# Loading a serialized model
model = load_model(kerasModels['dnn'])

In [3]:
# New dataset for forecasting
data = {
    'Location': ["ITO, Delhi", "Mumbai"],
    'Year': [2023, 2023],
    'Month': [2, 9],
    'Day': [16, 23],
    'Hour': [14, 17],
    'PM2.5': [43, 500],
    'PM10': [78, 480],
    'O3': [26, 91],
    'CO': [258, 78],
    'SO2': [10, 17],
    'NO2': [17, 47]  # Added missing value for test
}
new_data = pd.DataFrame(data)
data = pd.DataFrame(data)
new_data = data[['Location', 'Day', 'Hour', 'PM2.5', 'PM10', 'O3', 'CO', 'SO2', 'NO2']].copy()

In [4]:
# Label encoding
le = LabelEncoder()
new_data['Location'] = le.fit_transform(new_data['Location'])
new_data['Hour'] = le.fit_transform(new_data['Hour'])

# Data pre-processing
new_data.fillna(new_data.mean(), inplace=True)  # Заповнення пропущених значень середніми

# Convert hours to numeric format
new_data['Hour'] = new_data['Hour'].astype(str).str.split(":").str[0].astype(int)

# Feature selection
X_new = new_data[['Location','Day', 'Hour', 'PM2.5', 'PM10', 'O3', 'CO', 'SO2', 'NO2']]

In [5]:
print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │         2,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 131,333 (513.02 KB)

 Trainable params: 43,777 (171.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 87,556 (342.02 KB)

None


In [6]:
# Data scaling (as in the training model)
scaler = MinMaxScaler()
X_new_scaled = scaler.fit_transform(X_new)

In [7]:
# Prognostication
#X_new_scaled = np.expand_dims(X_new_scaled, axis=1) # LSTM only!!!
predictions = model.predict(X_new_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step


In [8]:
print(predictions)

[[0.05207454]
 [0.583851  ]]
